In [1]:
%reload_ext autoreload
%autoreload 2
%cd ../

/home/hazzu/Code/thesis


In [2]:
import cv2
import numpy as np

def visualize_result(image1: cv2.typing.MatLike, face1: list[float], image2: cv2.typing.MatLike, face2: list[float], distance: float, is_match: bool):
    drawed_image1 = image1.copy()
    drawed_image2 = image2.copy()
    
    x1, y1, w1, h1 = map(int, face1[:4])
    x2, y2, w2, h2 = map(int, face2[:4])
    
    color = (0, 0, 255)
    if is_match:
        color = (0, 255, 0)
    
    drawed_image1 = cv2.rectangle(drawed_image1, (x1, y1), (x1 + w1, y1 + h1), color, 2)
    drawed_image2 = cv2.rectangle(drawed_image2, (x2, y2), (x2 + w2, y2 + h2), color, 2)
    drawed_image1 = cv2.putText(drawed_image1, f"Distance: {distance}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
    
    h1, w1 = drawed_image1.shape[:2]
    h2, w2 = drawed_image2.shape[:2]
    vis = np.zeros((max(h1, h2), w1 + w2, 3), np.uint8)
    vis[:h1, :w1, :3] = drawed_image1
    vis[:h2, w1 : w1 + w2, :3] = drawed_image2
    
    return vis

# 1. YuNet + SFace


In [11]:
import os, csv
from tqdm import tqdm
from lib.FaceDetector import YuNetDetector
from lib.sface import SFace


CARDS_DIR = "datasets/cards"
SELFIES_DIR = "datasets/selfies_20241030"

cards_name = os.listdir(CARDS_DIR)
selfies_name = os.listdir(SELFIES_DIR)

yunet = YuNetDetector()
embedder = SFace(modelPath="weights/face_recognition_sface_2021dec.onnx")

yunet_name = []
yunet_distance = []
yunet_is_match = []

for card_name in tqdm(cards_name):
    card_path = os.path.join(CARDS_DIR, card_name)
    student_id = card_name.split("_")[0]
    
    # Find selfies of student
    selfies_of_student = list(filter(lambda x: student_id in x, selfies_name))
    if len(selfies_of_student) == 0:
        continue
    selfies_of_student = selfies_of_student[0]
    selfies_of_student_path = os.path.join(SELFIES_DIR, selfies_of_student)
    
    # Detect face in card
    card_image = cv2.imread(card_path)
    face_in_card, scale = yunet.detect_single_multiscale(card_image)
    if face_in_card is None:
        print(f"Cannot detect face in card {card_name}")
        continue
    scaled_card_image = cv2.resize(
        card_image, (int(scale * card_image.shape[1]), int(scale * card_image.shape[0]))
    )
    
    for selfie_name in os.listdir(selfies_of_student_path):
        selfie_image = cv2.imread(os.path.join(selfies_of_student_path, selfie_name))

        # Detect face in selfie
        face_in_selfie, scale2 = yunet.detect_single_multiscale(selfie_image)
        if face_in_selfie is None:
            print(f"Cannot detect face in selfie {os.path.join(selfies_of_student_path, selfie_name)}")
            continue
        scaled_selfie_image = cv2.resize(
            selfie_image,
            (int(scale2 * selfie_image.shape[1]), int(scale2 * selfie_image.shape[0])),
        )
        
        distance, is_match = embedder.match(
            scaled_card_image, face_in_card, scaled_selfie_image, face_in_selfie
        )
        
        vis = visualize_result(scaled_card_image, face_in_card, scaled_selfie_image, face_in_selfie, distance, is_match)
        os.makedirs(f"results/yunet-sface/{student_id}", exist_ok=True)
        cv2.imwrite(f"results/yunet-sface/{student_id}/{selfie_name}", vis)
        
        yunet_name.append(f"{student_id}_{selfie_name}")
        yunet_distance.append(distance)
        yunet_is_match.append(is_match)
        
fields = ["name", "distance", "is_match"]
rows = zip(yunet_name, yunet_distance, yunet_is_match)
filepath = "results/yunet-sface/result.csv"
with open(filepath, "w") as f:
    csvwriter = csv.writer(f)
    csvwriter.writerow(fields)
    csvwriter.writerows(rows)

100%|██████████| 314/314 [00:26<00:00, 11.77it/s]
